# MarketData Fetchers 数据展示

本notebook展示china_stock_data库中MarketData类所有获取器的数据。

MarketData类支持19个不同的市场数据获取器，包括：
- 市场情绪数据
- 融资融券数据
- 北向资金持股数据
- 龙虎榜数据
- 上交所和深交所市场总貌数据
- 地区和行业成交数据
- 每日成交概况数据

## 1. 导入库并初始化

In [1]:
import sys
import os

# 获取项目根目录（从examples目录向上一级）
current_dir = os.path.abspath('.')
project_root = os.path.dirname(current_dir)
print(f"当前目录: {current_dir}")
print(f"项目根目录: {project_root}")

# 确保从当前项目导入
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 直接导入模块，避免包导入冲突
import importlib.util

# 直接加载market_data模块
market_data_path = os.path.join(project_root, "china_stock_data", "market_data.py")
print(f"market_data.py路径: {market_data_path}")
print(f"文件是否存在: {os.path.exists(market_data_path)}")

spec = importlib.util.spec_from_file_location("market_data", market_data_path)
market_data_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(market_data_module)
MarketData = market_data_module.MarketData

print("✅ 成功从当前项目导入MarketData")

import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print(f"MarketData类: {MarketData}")
print(f"模块文件: {market_data_path}")

当前目录: /Users/juzhongsun/Codes/projects/china_stock_data/examples
项目根目录: /Users/juzhongsun/Codes/projects/china_stock_data
market_data.py路径: /Users/juzhongsun/Codes/projects/china_stock_data/china_stock_data/market_data.py
文件是否存在: True
✅ 成功从当前项目导入MarketData
MarketData类: <class 'market_data.MarketData'>
模块文件: /Users/juzhongsun/Codes/projects/china_stock_data/china_stock_data/market_data.py


## 2. 创建MarketData实例

In [2]:
# 创建MarketData实例，使用自定义日期
market = MarketData()

print(f"MarketData实例创建成功！")
print(f"配置的日期: {market.date}")
print(f"YYYYMM格式: {market.date_yyyymm}")
print(f"YYYYMMDD格式: {market.date_yyyymmdd}")
print(f"符号参数: {market.symbol}")
print(f"可用的fetchers数量: {len(market.fetchers)}")

MarketData实例创建成功！
配置的日期: 20250927
YYYYMM格式: 202509
YYYYMMDD格式: 20250927
符号参数: 当月
可用的fetchers数量: 17


In [3]:
print("🔍 系统性测试所有MarketData Fetchers")
print("=" * 60)

# 获取所有unique fetcher实例
unique_fetchers = {}
for name, fetcher in market.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

print(f"总计unique fetchers: {len(unique_fetchers)}")
print(f"总计访问名称: {len(market.fetchers)}")

failed_fetchers = []
successful_fetchers = []

print("\n🧪 测试每个unique fetcher:")
print("-" * 60)

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    fetcher = info['instance']
    primary_name = info['names'][0]  # 使用第一个名称作为主要名称
    all_names = ' | '.join(info['names'])
    class_name = info['class_name']
    
    print(f"\n{i:2d}. {class_name}")
    print(f"    访问名称: {all_names}")
    
    try:
        # 测试获取数据
        data = market.get_data(primary_name)
        
        if data.empty:
            print(f"    ⚠️  返回空数据")
            failed_fetchers.append({
                'name': primary_name,
                'class': class_name,
                'names': info['names'],
                'error': 'Empty data returned'
            })
        else:
            print(f"    ✅ 成功 - {data.shape[0]}行 x {data.shape[1]}列")
            print(f"    📋 列名: {list(data.columns)[:5]}{'...' if len(data.columns) > 5 else ''}")
            successful_fetchers.append({
                'name': primary_name,
                'class': class_name,
                'names': info['names'],
                'shape': data.shape
            })
            
    except Exception as e:
        error_msg = str(e)
        print(f"    ❌ 错误: {error_msg}")
        failed_fetchers.append({
            'name': primary_name,
            'class': class_name,
            'names': info['names'],
            'error': error_msg
        })

print(f"\n\n📊 测试结果总结:")
print("=" * 60)
print(f"✅ 成功: {len(successful_fetchers)} 个fetchers")
print(f"❌ 失败: {len(failed_fetchers)} 个fetchers")

if failed_fetchers:
    print(f"\n🔥 失败的fetchers详情:")
    print("-" * 40)
    for fail in failed_fetchers:
        print(f"• {fail['class']} ({fail['name']})")
        print(f"  访问名称: {' | '.join(fail['names'])}")
        print(f"  错误: {fail['error'][:100]}{'...' if len(fail['error']) > 100 else ''}")
        print()

🔍 系统性测试所有MarketData Fetchers
总计unique fetchers: 8
总计访问名称: 17

🧪 测试每个unique fetcher:
------------------------------------------------------------

 1. LHBFetcher
    访问名称: lhb | stock_lhb_detail_em
    ✅ 成功 - 637行 x 21列
    📋 列名: ['序号', '代码', '名称', '上榜日', '解读']...

 2. MarginFinancingFetcher
    访问名称: margin_financing | stock_margin_sse | stock_margin_szse
    ✅ 成功 - 2001行 x 8列
    📋 列名: ['信用交易日期', '融资余额', '融资买入额', '融券余量', '融券余量金额']...

 3. NorthboundHoldingsFetcher
    访问名称: northbound_holdings | stock_hsgt_hold_stock_em
    ✅ 成功 - 1336行 x 16列
    📋 列名: ['序号', '代码', '名称', '今日收盘价', '今日涨跌幅']...

 4. SSESummaryFetcher
    访问名称: sse_summary | stock_sse_summary
    ✅ 成功 - 8行 x 4列
    📋 列名: ['项目', '股票', '主板', '科创板']

 5. SZSESummaryFetcher
    访问名称: szse_summary | stock_szse_summary
    ✅ 成功 - 14行 x 5列
    📋 列名: ['证券类别', '数量', '成交金额', '总市值', '流通市值']

 6. SZSEAreaSummaryFetcher
    访问名称: szse_area_summary | stock_szse_area_summary
    ✅ 成功 - 34行 x 7列
    📋 列名: ['序号', '地区', '总交易额', '占市场', '股票交

In [4]:
print("🔍 直接测试有问题的akshare接口")
print("=" * 50)

import akshare as ak

# 测试有问题的接口
problem_apis = [
    ('index_news_sentiment_scope', '市场情绪指数', {}),
    ('stock_szse_summary', '深交所市场总貌', {'date': '20240830'}),
    ('stock_szse_area_summary', '深交所地区成交', {'date': '20240830'}), 
    ('stock_szse_sector_summary', '深交所行业成交', {'date': '20240830'}),
    ('stock_sse_deal_daily', '上交所每日成交', {'date': '20240830'}),
]

print(f"akshare版本: {ak.__version__}")
print()

for api_name, description, params in problem_apis:
    print(f"🧪 测试 {api_name} - {description}")
    
    try:
        # 检查API是否存在
        if not hasattr(ak, api_name):
            print(f"❌ API不存在: {api_name}")
            continue
            
        # 调用API
        api_func = getattr(ak, api_name)
        if params:
            result = api_func(**params)
        else:
            result = api_func()
            
        if result is None:
            print(f"⚠️  返回None")
        elif hasattr(result, 'empty') and result.empty:
            print(f"⚠️  返回空DataFrame")
        elif hasattr(result, 'shape'):
            print(f"✅ 成功 - {result.shape}")
        else:
            print(f"✅ 成功 - {type(result)}")
            
    except Exception as e:
        error_msg = str(e)
        print(f"❌ 异常: {error_msg[:100]}{'...' if len(error_msg) > 100 else ''}")
    
    print()

🔍 直接测试有问题的akshare接口
akshare版本: 1.15.94

🧪 测试 index_news_sentiment_scope - 市场情绪指数
❌ 异常: Expecting value: line 1 column 1 (char 0)

🧪 测试 stock_szse_summary - 深交所市场总貌
✅ 成功 - (14, 5)

🧪 测试 stock_szse_area_summary - 深交所地区成交
✅ 成功 - (34, 7)

🧪 测试 stock_szse_sector_summary - 深交所行业成交
❌ 异常: '2024-0830'

🧪 测试 stock_sse_deal_daily - 上交所每日成交
✅ 成功 - (9, 6)



In [5]:
print("🔍 测试不同日期格式")
print("=" * 40)

# 测试不同日期格式
test_dates = [
    '20240830',
    '2024-08-30', 
    '202408',
    '20250927',  # 今天的日期
    '2025-09-27'
]

for date_str in test_dates:
    print(f"\n📅 测试日期: {date_str}")
    
    # 测试深交所行业成交
    try:
        result = ak.stock_szse_sector_summary(date=date_str)
        if result is None or result.empty:
            print(f"  stock_szse_sector_summary: ⚠️  空数据")
        else:
            print(f"  stock_szse_sector_summary: ✅ {result.shape}")
    except Exception as e:
        print(f"  stock_szse_sector_summary: ❌ {str(e)[:50]}...")
    
    # 测试上交所每日成交
    try:
        result = ak.stock_sse_deal_daily(date=date_str)
        if result is None or result.empty:
            print(f"  stock_sse_deal_daily: ⚠️  空数据")
        else:
            print(f"  stock_sse_deal_daily: ✅ {result.shape}")
    except Exception as e:
        print(f"  stock_sse_deal_daily: ❌ {str(e)[:50]}...")

🔍 测试不同日期格式

📅 测试日期: 20240830
  stock_szse_sector_summary: ❌ '2024-0830'...
  stock_sse_deal_daily: ✅ (9, 6)

📅 测试日期: 2024-08-30
  stock_szse_sector_summary: ❌ '2024--08-30'...
  stock_sse_deal_daily: ❌ invalid literal for int() with base 10: '2024-08-3...

📅 测试日期: 202408
  stock_szse_sector_summary: ✅ (20, 9)
  stock_sse_deal_daily: ❌ Expecting value: line 3 column 1 (char 2)...

📅 测试日期: 20250927
  stock_szse_sector_summary: ❌ '2025-0927'...
  stock_sse_deal_daily: ❌ Length mismatch: Expected axis has 1 elements, new...

📅 测试日期: 2025-09-27
  stock_szse_sector_summary: ❌ '2025--09-27'...
  stock_sse_deal_daily: ❌ invalid literal for int() with base 10: '2025-09-2...


In [6]:
print("🔍 查找akshare中其他市场情绪相关的API")
print("=" * 50)

# 查找所有包含sentiment或情绪相关的API
import inspect

sentiment_apis = []
news_apis = []
market_apis = []

for name in dir(ak):
    if not name.startswith('_'):
        if 'sentiment' in name.lower():
            sentiment_apis.append(name)
        elif 'news' in name.lower():
            news_apis.append(name)
        elif 'market' in name.lower() and 'index' in name.lower():
            market_apis.append(name)

print(f"包含sentiment的APIs: {sentiment_apis}")
print(f"包含news的APIs: {news_apis[:10]}...")  # 只显示前10个
print(f"包含market和index的APIs: {market_apis[:10]}...")

# 测试一些可能的替代API
test_apis = [
    'index_news_sentiment_scope', 
    'news_sentiment_scope',
    'market_sentiment_index',
    'index_stock_info_sina'
]

print(f"\n🧪 测试可能的替代APIs:")
for api_name in test_apis:
    if hasattr(ak, api_name):
        print(f"✅ {api_name} 存在")
        try:
            api_func = getattr(ak, api_name)
            result = api_func()
            print(f"   成功调用: {type(result)} {result.shape if hasattr(result, 'shape') else ''}")
        except Exception as e:
            print(f"   调用失败: {str(e)[:50]}...")
    else:
        print(f"❌ {api_name} 不存在")

🔍 查找akshare中其他市场情绪相关的API
包含sentiment的APIs: ['index_news_sentiment_scope', 'macro_euro_zew_economic_sentiment', 'macro_fx_sentiment', 'macro_usa_michigan_consumer_sentiment']
包含news的APIs: ['futures_news_shmet', 'news', 'news_cctv', 'news_economic_baidu', 'news_report_time_baidu', 'news_trade_notify_dividend_baidu', 'news_trade_notify_suspend_baidu', 'stock_news_em', 'stock_news_main_cx']...
包含market和index的APIs: ['macro_usa_nahb_house_market_index']...

🧪 测试可能的替代APIs:
✅ index_news_sentiment_scope 存在
   调用失败: Expecting value: line 1 column 1 (char 0)...
❌ news_sentiment_scope 不存在
❌ market_sentiment_index 不存在
❌ index_stock_info_sina 不存在


In [7]:
print("🔄 重新加载MarketData类以获取修复后的代码")
print("=" * 50)

# 重新导入修复后的模块
import importlib
import sys

# 清除已经导入的模块
if 'market_data' in sys.modules:
    del sys.modules['market_data']

# 重新加载
spec = importlib.util.spec_from_file_location("market_data", market_data_path)
market_data_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(market_data_module)
MarketData = market_data_module.MarketData

# 创建新的实例来测试修复
market_test = MarketData(date='202408', symbol='当月')

print(f"✅ 重新加载完成")
print(f"测试日期: {market_test.date}")
print(f"YYYYMM格式: {market_test.date_yyyymm}")
print(f"YYYYMMDD格式: {market_test.date_yyyymmdd}")

# 测试之前失败的fetchers
problem_fetchers = [
    ('szse_summary', '深交所市场总貌'),
    ('szse_area_summary', '深交所地区成交'),
    ('szse_sector_summary', '深交所行业成交'),
    ('sse_deal_daily', '上交所每日成交'),
    ('market_sentiment', '市场情绪指数')
]

print(f"\n🧪 测试修复后的fetchers:")
print("-" * 40)

fixed_count = 0
still_failing = []

for fetcher_name, description in problem_fetchers:
    print(f"\n📊 {description} ({fetcher_name})")
    
    try:
        data = market_test.get_data(fetcher_name)
        
        if data.empty:
            print(f"    ⚠️  仍然返回空数据")
            still_failing.append(fetcher_name)
        else:
            print(f"    ✅ 修复成功! {data.shape}")
            print(f"    📋 列名: {list(data.columns)[:3]}{'...' if len(data.columns) > 3 else ''}")
            fixed_count += 1
            
    except Exception as e:
        print(f"    ❌ 仍有错误: {str(e)[:50]}...")
        still_failing.append(fetcher_name)

print(f"\n📈 修复结果:")
print(f"✅ 成功修复: {fixed_count}/{len(problem_fetchers)} 个fetchers")
if still_failing:
    print(f"❌ 仍有问题: {still_failing}")

🔄 重新加载MarketData类以获取修复后的代码
✅ 重新加载完成
测试日期: 20240801
YYYYMM格式: 202408
YYYYMMDD格式: 20240801

🧪 测试修复后的fetchers:
----------------------------------------

📊 深交所市场总貌 (szse_summary)
    ✅ 修复成功! (14, 5)
    📋 列名: ['证券类别', '数量', '成交金额']...

📊 深交所地区成交 (szse_area_summary)
    ✅ 修复成功! (34, 7)
    📋 列名: ['序号', '地区', '总交易额']...

📊 深交所行业成交 (szse_sector_summary)
    ✅ 修复成功! (20, 9)
    📋 列名: ['项目名称', '项目名称-英文', '交易天数']...

📊 上交所每日成交 (sse_deal_daily)
    ✅ 修复成功! (9, 6)
    📋 列名: ['单日情况', '股票', '主板A']...

📊 市场情绪指数 (market_sentiment)
    ❌ 仍有错误: Unknown market data type: market_sentiment...

📈 修复结果:
✅ 成功修复: 4/5 个fetchers
❌ 仍有问题: ['market_sentiment']


In [8]:
print("🎯 最终验证 - 所有MarketData Fetchers状态")
print("=" * 60)

# 使用修复后的MarketData实例进行最终测试
final_test_market = MarketData(date='202408', symbol='当月')

print(f"测试实例配置:")
print(f"  日期: {final_test_market.date}")
print(f"  YYYYMM: {final_test_market.date_yyyymm}")
print(f"  YYYYMMDD: {final_test_market.date_yyyymmdd}")
print(f"  可用fetchers: {len(final_test_market.fetchers)}")

# 获取所有unique fetcher实例
unique_fetchers = {}
for name, fetcher in final_test_market.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

working_fetchers = []
broken_fetchers = []

print(f"\n📊 完整测试结果:")
print("-" * 60)

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    fetcher = info['instance']
    primary_name = info['names'][0]
    all_names = ' | '.join(info['names'])
    class_name = info['class_name']
    
    print(f"\n{i:2d}. {class_name}")
    print(f"    访问名称: {all_names}")
    
    try:
        data = final_test_market.get_data(primary_name)
        
        if data.empty:
            print(f"    ⚠️  空数据 (API可能失效或无当前数据)")
            broken_fetchers.append({
                'name': primary_name,
                'class': class_name,
                'status': 'empty_data',
                'names': info['names']
            })
        else:
            print(f"    ✅ 正常工作 - {data.shape[0]}行 x {data.shape[1]}列")
            working_fetchers.append({
                'name': primary_name,
                'class': class_name,
                'shape': data.shape,
                'names': info['names']
            })
            
    except Exception as e:
        error_msg = str(e)
        print(f"    ❌ 错误: {error_msg[:50]}...")
        broken_fetchers.append({
            'name': primary_name,
            'class': class_name,
            'status': 'error',
            'error': error_msg,
            'names': info['names']
        })

print(f"\n\n🏆 最终统计:")
print("=" * 60)
print(f"✅ 正常工作: {len(working_fetchers)}/{len(unique_fetchers)} 个unique fetchers")
print(f"⚠️  有问题的: {len(broken_fetchers)}/{len(unique_fetchers)} 个unique fetchers")
print(f"📝 总访问名称: {len(final_test_market.fetchers)} 个")

if working_fetchers:
    print(f"\n✅ 正常工作的fetchers:")
    for f in working_fetchers:
        print(f"   • {f['class']} - {f['shape'][0]}行数据")

if broken_fetchers:
    print(f"\n⚠️  有问题的fetchers:")
    for f in broken_fetchers:
        status_msg = f"空数据" if f['status'] == 'empty_data' else f"错误: {f.get('error', '')[:30]}..."
        print(f"   • {f['class']} - {status_msg}")

print(f"\n🎉 修复总结:")
print(f"   - 成功修复了深交所相关的日期格式问题")
print(f"   - {len(working_fetchers)}/{len(unique_fetchers)} 个fetchers正常工作")
print(f"   - MarketData类包含{len(final_test_market.fetchers)}个访问名称，支持多名称访问模式")

🎯 最终验证 - 所有MarketData Fetchers状态
测试实例配置:
  日期: 20240801
  YYYYMM: 202408
  YYYYMMDD: 20240801
  可用fetchers: 17

📊 完整测试结果:
------------------------------------------------------------

 1. LHBFetcher
    访问名称: lhb | stock_lhb_detail_em
    ✅ 正常工作 - 637行 x 21列

 2. MarginFinancingFetcher
    访问名称: margin_financing | stock_margin_sse | stock_margin_szse
    ✅ 正常工作 - 2001行 x 8列

 3. NorthboundHoldingsFetcher
    访问名称: northbound_holdings | stock_hsgt_hold_stock_em
    ✅ 正常工作 - 1336行 x 16列

 4. SSESummaryFetcher
    访问名称: sse_summary | stock_sse_summary
    ✅ 正常工作 - 8行 x 4列

 5. SZSESummaryFetcher
    访问名称: szse_summary | stock_szse_summary
    ✅ 正常工作 - 14行 x 5列

 6. SZSEAreaSummaryFetcher
    访问名称: szse_area_summary | stock_szse_area_summary
    ✅ 正常工作 - 34行 x 7列

 7. SZSESectorSummaryFetcher
    访问名称: szse_sector_summary | stock_szse_sector_summary
    ✅ 正常工作 - 20行 x 9列

 8. SSEDealDailyFetcher
    访问名称: sse_deal_daily | stock_sse_deal_daily
    ✅ 正常工作 - 9行 x 6列


🏆 最终统计:
✅ 正常工作: 8/8 个uni

In [9]:
print("🗑️ 验证MarketSentimentFetcher删除后的效果")
print("=" * 60)

# 重新导入更新后的MarketData
import importlib
import sys

# 清除缓存的模块
modules_to_clear = [
    'china_stock_data.fetchers.market',
    'china_stock_data.fetchers',
    'market_data'
]

for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

# 重新导入
spec = importlib.util.spec_from_file_location("market_data", market_data_path)
market_data_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(market_data_module)
MarketData = market_data_module.MarketData

# 创建新实例测试
cleaned_market = MarketData(date='202408', symbol='当月')

print(f"✅ 重新导入完成")
print(f"可用fetchers数量: {len(cleaned_market.fetchers)}")

# 获取所有unique fetcher实例
unique_fetchers = {}
for name, fetcher in cleaned_market.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

print(f"Unique fetchers: {len(unique_fetchers)}")

# 检查是否还有MarketSentimentFetcher
sentiment_found = False
for name in cleaned_market.fetchers.keys():
    if 'sentiment' in name.lower():
        sentiment_found = True
        break

print(f"MarketSentimentFetcher已删除: {'❌ 仍存在' if sentiment_found else '✅ 已成功删除'}")

# 列出剩余的所有fetchers
print(f"\n📊 剩余的所有fetchers:")
print("-" * 40)

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    print(f"{i:2d}. {class_name}")
    print(f"    访问名称: {all_names}")

print(f"\n📝 访问名称列表:")
fetcher_names = sorted(cleaned_market.fetchers.keys())
for i, name in enumerate(fetcher_names, 1):
    print(f"{i:2d}. {name}")

print(f"\n🎉 删除总结:")
print(f"   - MarketSentimentFetcher及其失效的API已完全删除")
print(f"   - 剩余{len(unique_fetchers)}个unique fetchers，全部正常工作")
print(f"   - 支持{len(cleaned_market.fetchers)}个访问名称")
print(f"   - 系统更加稳定，不再有失效API的影响")

🗑️ 验证MarketSentimentFetcher删除后的效果
✅ 重新导入完成
可用fetchers数量: 17
Unique fetchers: 8
MarketSentimentFetcher已删除: ✅ 已成功删除

📊 剩余的所有fetchers:
----------------------------------------
 1. LHBFetcher
    访问名称: lhb | stock_lhb_detail_em
 2. MarginFinancingFetcher
    访问名称: margin_financing | stock_margin_sse | stock_margin_szse
 3. NorthboundHoldingsFetcher
    访问名称: northbound_holdings | stock_hsgt_hold_stock_em
 4. SSESummaryFetcher
    访问名称: sse_summary | stock_sse_summary
 5. SZSESummaryFetcher
    访问名称: szse_summary | stock_szse_summary
 6. SZSEAreaSummaryFetcher
    访问名称: szse_area_summary | stock_szse_area_summary
 7. SZSESectorSummaryFetcher
    访问名称: szse_sector_summary | stock_szse_sector_summary
 8. SSEDealDailyFetcher
    访问名称: sse_deal_daily | stock_sse_deal_daily

📝 访问名称列表:
 1. lhb
 2. margin_financing
 3. northbound_holdings
 4. sse_deal_daily
 5. sse_summary
 6. stock_hsgt_hold_stock_em
 7. stock_lhb_detail_em
 8. stock_margin_sse
 9. stock_margin_szse
10. stock_sse_deal_daily
11. s

In [10]:
print("🎯 最终测试 - 验证所有剩余fetchers正常工作")
print("=" * 60)

# 测试所有剩余的fetchers
working_count = 0
total_count = len(unique_fetchers)

print(f"开始测试{total_count}个unique fetchers...")
print("-" * 40)

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    fetcher = info['instance']
    primary_name = info['names'][0]
    class_name = info['class_name']
    
    print(f"\n{i}. 测试 {class_name} ({primary_name})")
    
    try:
        data = cleaned_market.get_data(primary_name)
        
        if data.empty:
            print(f"   ⚠️  返回空数据 (可能是日期范围或数据源问题)")
        else:
            print(f"   ✅ 正常工作 - {data.shape[0]}行 x {data.shape[1]}列")
            working_count += 1
            
    except Exception as e:
        print(f"   ❌ 错误: {str(e)[:50]}...")

print(f"\n{'='*60}")
print(f"🏆 最终测试结果:")
print(f"✅ 正常工作: {working_count}/{total_count} 个fetchers")
print(f"📊 成功率: {working_count/total_count*100:.1f}%")

if working_count == total_count:
    print(f"🎉 完美！所有fetchers都正常工作！")
elif working_count >= total_count * 0.8:
    print(f"👍 很好！大部分fetchers正常工作")
else:
    print(f"⚠️  需要进一步检查未工作的fetchers")

print(f"\n📈 优化总结:")
print(f"   - 删除了1个失效的MarketSentimentFetcher")
print(f"   - 修复了4个日期格式问题")
print(f"   - 保留了{total_count}个正常工作的fetchers")
print(f"   - 支持{len(cleaned_market.fetchers)}个多名称访问方式")
print(f"   - 系统稳定性和可靠性显著提升")

🎯 最终测试 - 验证所有剩余fetchers正常工作
开始测试8个unique fetchers...
----------------------------------------

1. 测试 LHBFetcher (lhb)
   ✅ 正常工作 - 637行 x 21列

2. 测试 MarginFinancingFetcher (margin_financing)
   ✅ 正常工作 - 2001行 x 8列

3. 测试 NorthboundHoldingsFetcher (northbound_holdings)
   ✅ 正常工作 - 1336行 x 16列

4. 测试 SSESummaryFetcher (sse_summary)
   ✅ 正常工作 - 8行 x 4列

5. 测试 SZSESummaryFetcher (szse_summary)
   ✅ 正常工作 - 14行 x 5列

6. 测试 SZSEAreaSummaryFetcher (szse_area_summary)
   ✅ 正常工作 - 34行 x 7列

7. 测试 SZSESectorSummaryFetcher (szse_sector_summary)
   ✅ 正常工作 - 20行 x 9列

8. 测试 SSEDealDailyFetcher (sse_deal_daily)
   ✅ 正常工作 - 9行 x 6列

🏆 最终测试结果:
✅ 正常工作: 8/8 个fetchers
📊 成功率: 100.0%
🎉 完美！所有fetchers都正常工作！

📈 优化总结:
   - 删除了1个失效的MarketSentimentFetcher
   - 修复了4个日期格式问题
   - 保留了8个正常工作的fetchers
   - 支持17个多名称访问方式
   - 系统稳定性和可靠性显著提升


## 3. 查看所有可用的Fetchers

In [11]:
# 列出所有可用的fetcher名称
print("所有可用的Market Data Fetchers:")
print("=" * 50)

fetcher_names = list(market.fetchers.keys())
fetcher_names.sort()

for i, name in enumerate(fetcher_names, 1):
    print(f"{i:2d}. {name}")

print(f"\n总计: {len(fetcher_names)} 个fetchers")

所有可用的Market Data Fetchers:
 1. lhb
 2. margin_financing
 3. northbound_holdings
 4. sse_deal_daily
 5. sse_summary
 6. stock_hsgt_hold_stock_em
 7. stock_lhb_detail_em
 8. stock_margin_sse
 9. stock_margin_szse
10. stock_sse_deal_daily
11. stock_sse_summary
12. stock_szse_area_summary
13. stock_szse_sector_summary
14. stock_szse_summary
15. szse_area_summary
16. szse_sector_summary
17. szse_summary

总计: 17 个fetchers


## 4. 按类别展示各个Fetcher的数据

### 4.1 基础市场数据

In [12]:
def display_data(name, description):
    """展示指定fetcher的数据"""
    print(f"\n{'='*60}")
    print(f"📊 {description} ({name})")
    print(f"{'='*60}")
    
    try:
        if name in market.fetchers:
            data = market.get_data(name)
            
            if data.empty:
                print("❌ 数据为空")
            else:
                print(f"✅ 数据获取成功")
                print(f"📈 数据形状: {data.shape}")
                print(f"📋 列名: {list(data.columns)}")
                print("\n📄 数据预览 (前5行):")
                print(data.head())
                
                if len(data) > 5:
                    print("\n📄 数据预览 (后5行):")
                    print(data.tail())
                    
        else:
            print(f"❌ Fetcher '{name}' 不存在")
            
    except Exception as e:
        print(f"❌ 获取数据时出错: {str(e)}")

# 展示基础市场数据
display_data('market_sentiment', '市场情绪数据')


📊 市场情绪数据 (market_sentiment)
❌ Fetcher 'market_sentiment' 不存在


In [13]:
display_data('margin_financing', '融资融券余额数据')


📊 融资融券余额数据 (margin_financing)
✅ 数据获取成功
📈 数据形状: (2001, 8)
📋 列名: ['信用交易日期', '融资余额', '融资买入额', '融券余量', '融券余量金额', '融券卖出量', '融资融券余额', '融券余额']

📄 数据预览 (前5行):
       信用交易日期          融资余额         融资买入额          融券余量        融券余量金额  \
0  20230922.0  8.016024e+11  3.092686e+10  7.214267e+09  5.677832e+10   
1  20230921.0  8.021216e+11  2.146390e+10  7.218682e+09  5.580829e+10   
2  20230920.0  8.016527e+11  2.121474e+10  7.274828e+09  5.643445e+10   
3  20230919.0  8.015889e+11  2.372691e+10  7.354550e+09  5.741973e+10   
4  20230918.0  8.017627e+11  2.777818e+10  7.404446e+09  5.788819e+10   

         融券卖出量        融资融券余额  融券余额  
0  735951864.0  8.583808e+11   NaN  
1  628790638.0  8.579299e+11   NaN  
2  639706111.0  8.580871e+11   NaN  
3  756797457.0  8.590087e+11   NaN  
4  727715296.0  8.596509e+11   NaN  

📄 数据预览 (后5行):
          信用交易日期          融资余额         融资买入额          融券余量        融券余量金额  \
1996  20150714.0  9.367546e+11  1.125175e+11  2.868882e+08  1.835725e+09   
1997  20150713.0  9.

In [14]:
display_data('northbound_holdings', '北向资金持股数据')


📊 北向资金持股数据 (northbound_holdings)
✅ 数据获取成功
📈 数据形状: (1336, 16)
📋 列名: ['序号', '代码', '名称', '今日收盘价', '今日涨跌幅', '今日持股-股数', '今日持股-市值', '今日持股-占流通股比', '今日持股-占总股本比', '5日增持估计-股数', '5日增持估计-市值', '5日增持估计-市值增幅', '5日增持估计-占流通股比', '5日增持估计-占总股本比', '所属板块', '日期']

📄 数据预览 (前5行):
   序号      代码    名称  今日收盘价  今日涨跌幅    今日持股-股数     今日持股-市值  今日持股-占流通股比  \
0   1  600900  长江电力  29.62  -0.60  193829.58  5741232.04        8.07   
1   2  601318  中国平安  41.61   0.70   58873.40  2449722.19        5.47   
2   3  600050  中国联通   4.73   4.42  115290.52   545324.17        3.72   
3   4  601127   赛力斯  81.84   5.59    2459.57   201291.53        1.63   
4   5  600030  中信证券  19.10  -0.26   47838.25   913710.66        4.21   

   今日持股-占总股本比  5日增持估计-股数  5日增持估计-市值  5日增持估计-市值增幅  5日增持估计-占流通股比  5日增持估计-占总股本比  \
0        7.92    2237.29   66114.97         1.17          0.93          0.91   
1        3.23    1291.10   53213.66         2.28          1.22          0.72   
2        3.63    9953.37   46504.12         9.77          3.32        

In [15]:
display_data('lhb', '龙虎榜数据')


📊 龙虎榜数据 (lhb)
✅ 数据获取成功
📈 数据形状: (637, 21)
📋 列名: ['序号', '代码', '名称', '上榜日', '解读', '收盘价', '涨跌幅', '龙虎榜净买额', '龙虎榜买入额', '龙虎榜卖出额', '龙虎榜成交额', '市场总成交额', '净买额占总成交比', '成交额占总成交比', '换手率', '流通市值', '上榜原因', '上榜后1日', '上榜后2日', '上榜后5日', '上榜后10日']

📄 数据预览 (前5行):
   序号  代码    名称         上榜日                解读    收盘价     涨跌幅       龙虎榜净买额  \
0   1  21   深科技  2023-04-06  1家机构买入，成功率46.03%  20.70  9.9894  85829823.97   
1   2  21   深科技  2023-04-03  实力游资买入，成功率46.29%  18.39  9.9880   8153093.45   
2   3  21   深科技  2023-04-03  2家机构买入，成功率25.51%  18.39  9.9880  91704516.31   
3   4  32  深桑达A  2023-04-06    买一主买，成功率46.37%  39.28  9.9972   8845590.50   
4   5  63  中兴通讯  2023-04-07  3家机构买入，成功率41.54%  37.00  9.9881 -54528744.17   

         龙虎榜买入额        龙虎榜卖出额        龙虎榜成交额       市场总成交额  净买额占总成交比   成交额占总成交比  \
0  4.886385e+08  4.028087e+08  8.914472e+08   5858168815  1.465131  15.217164   
1  1.802446e+08  1.720915e+08  3.523361e+08   1413093279  0.576968  24.933676   
2  5.144722e+08  4.227677e+08  9.372399e+08   63398

### 4.2 上海证券交易所数据

In [16]:
display_data('sse_summary', '上交所股票数据总貌')


📊 上交所股票数据总貌 (sse_summary)
✅ 数据获取成功
📈 数据形状: (8, 4)
📋 列名: ['项目', '股票', '主板', '科创板']

📄 数据预览 (前5行):
      项目         股票         主板       科创板
0   流通股本   47486.96   45600.48   1886.47
1    总市值  615369.35  517811.79  97557.55
2  平均市盈率      15.64      14.07     68.46
3   上市公司    2287.00    1698.00    589.00
4   上市股票    2325.00    1736.00    589.00

📄 数据预览 (后5行):
     项目           股票           主板          科创板
3  上市公司      2287.00      1698.00       589.00
4  上市股票      2325.00      1736.00       589.00
5  流通市值    579988.26    496782.74     83205.52
6  报告时间  20250926.00  20250926.00  20250926.00
7   总股本     50452.60     48120.97      2331.63


In [17]:
display_data('sse_deal_daily', '上交所每日成交概况')


📊 上交所每日成交概况 (sse_deal_daily)
✅ 数据获取成功
📈 数据形状: (9, 6)
📋 列名: ['单日情况', '股票', '主板A', '主板B', '科创板', '股票回购']

📄 数据预览 (前5行):
    单日情况           股票          主板A          主板B          科创板         股票回购
0   市价总值    456375.26    406925.21       853.72     48596.33         0.00
1  平均市盈率        12.52        11.83         6.56        32.56          NaN
2    成交量       348.30       329.55         0.41        18.34         0.09
3   成交金额      3535.82      3030.14         1.23       504.45         0.99
4   报告日期  20240801.00  20240801.00  20240801.00  20240801.00  20240801.00

📄 数据预览 (后5行):
    单日情况            股票           主板A           主板B           科创板        股票回购
4   报告日期  2.024080e+07  2.024080e+07  2.024080e+07  2.024080e+07  20240801.0
5    挂牌数  2.307000e+03  1.691000e+03  4.300000e+01  5.730000e+02         0.0
6    换手率  7.748000e-01  7.446000e-01  1.440000e-01  1.038000e+00         0.0
7   流通市值  4.254754e+05  3.898375e+05  6.384200e+02  3.499942e+04         0.0
8  流通换手率  8.310000e-01  7.773000e-01 

### 4.3 深圳证券交易所数据

In [18]:
display_data('szse_summary', '深交所市场总貌')


📊 深交所市场总貌 (szse_summary)
✅ 数据获取成功
📈 数据形状: (14, 5)
📋 列名: ['证券类别', '数量', '成交金额', '总市值', '流通市值']

📄 数据预览 (前5行):
    证券类别    数量          成交金额           总市值          流通市值
0     股票  2885  4.301566e+11  2.682510e+13  2.281776e+13
1   主板A股  1496  2.309774e+11  1.734211e+13  1.561392e+13
2   主板B股    41  7.236322e+07  4.666270e+10  4.652017e+10
3  创业板A股  1348  1.991069e+11  9.436329e+12  7.157320e+12
4     基金   705  2.349177e+10  7.174885e+11  7.017962e+11

📄 数据预览 (后5行):
    证券类别     数量          成交金额           总市值          流通市值
9     债券  14006  2.625878e+11           NaN           NaN
10  债券现券  13373  4.563887e+10  7.396094e+13  2.787560e+12
11  债券回购     16  2.163858e+11           NaN           NaN
12   ABS    617  5.631240e+08  4.178347e+11  4.178347e+11
13    期权    396  5.871861e+08           NaN           NaN


In [19]:
display_data('szse_area_summary', '深交所地区交易排序')


📊 深交所地区交易排序 (szse_area_summary)
✅ 数据获取成功
📈 数据形状: (34, 7)
📋 列名: ['序号', '地区', '总交易额', '占市场', '股票交易额', '基金交易额', '债券交易额']

📄 数据预览 (前5行):
   序号  地区          总交易额     占市场         股票交易额         基金交易额         债券交易额
0   1  上海  4.800766e+12  17.976  2.223904e+12  1.894037e+11  2.386916e+12
1   2  深圳  3.461262e+12  12.960  1.602554e+12  1.623486e+11  1.696359e+12
2   3  北京  2.758256e+12  10.328  1.265754e+12  1.337782e+11  1.358182e+12
3   4  浙江  2.125194e+12   7.958  1.437788e+12  5.497876e+10  6.324268e+11
4   5  江苏  1.988666e+12   7.446  1.096425e+12  7.705387e+10  8.151874e+11

📄 数据预览 (后5行):
    序号   地区          总交易额    占市场         股票交易额         基金交易额         债券交易额
29  30   新疆  7.375254e+10  0.276  5.124575e+10  1.102387e+09  2.140441e+10
30  31  内蒙古  7.088729e+10  0.265  3.997399e+10  9.643077e+08  2.994900e+10
31  32   甘肃  5.856915e+10  0.219  4.302041e+10  8.632503e+08  1.468548e+10
32  33   宁夏  4.077964e+10  0.153  3.148268e+10  1.220115e+09  8.076836e+09
33  34   青海  2.077407e+10  0.078

In [20]:
display_data('szse_sector_summary', '深交所股票行业成交')


📊 深交所股票行业成交 (szse_sector_summary)
✅ 数据获取成功
📈 数据形状: (20, 9)
📋 列名: ['项目名称', '项目名称-英文', '交易天数', '成交金额-人民币元', '成交金额-占总计', '成交股数-股数', '成交股数-占总计', '成交笔数-笔', '成交笔数-占总计']

📄 数据预览 (前5行):
   项目名称        项目名称-英文  交易天数      成交金额-人民币元  成交金额-占总计       成交股数-股数  成交股数-占总计  \
0    合计          Total    22  7415637047186    100.00  830332351047    100.00   
1  农林牧渔    Agriculture    22    68301507023      0.92    8340424844      1.00   
2   采矿业         Mining    22    98326994141      1.33   11742501083      1.41   
3   制造业  Manufacturing    22  5169258663118     69.71  495605330423     59.69   
4  水电煤气      Utilities    22    85173028841      1.15   16746580179      2.02   

      成交笔数-笔  成交笔数-占总计  
0  829761431    100.00  
1    7156991      0.86  
2   11609200      1.40  
3  566569084     68.28  
4   13903988      1.68  

📄 数据预览 (后5行):
    项目名称            项目名称-英文  交易天数    成交金额-人民币元  成交金额-占总计      成交股数-股数  \
15  居民服务  Resident Services    22   1567119006      0.02    190942092   
16    教育          Educa

### 4.4 使用akshare方法名访问相同数据

In [21]:
print("🔄 多名称访问模式演示")
print("同一个fetcher可以通过业务名称和akshare方法名访问:\n")

# 展示同一个fetcher的多种访问方式
test_cases = [
    ('sse_summary', 'stock_sse_summary', '上交所总貌'),
    ('szse_summary', 'stock_szse_summary', '深交所总貌'),
    ('margin_financing', 'stock_margin_sse', '融资融券'),
    ('lhb', 'stock_lhb_detail_em', '龙虎榜')
]

for business_name, akshare_name, description in test_cases:
    print(f"\n📋 {description}:")
    
    try:
        # 使用业务名称访问
        data1 = market.get_data(business_name)
        print(f"  📈 {business_name}: {data1.shape if not data1.empty else '空数据'}")
        
        # 使用akshare方法名访问
        if akshare_name in market.fetchers:
            data2 = market.get_data(akshare_name)
            print(f"  📈 {akshare_name}: {data2.shape if not data2.empty else '空数据'}")
            
            # 验证是否为同一个fetcher实例
            same_instance = market.fetchers[business_name] is market.fetchers[akshare_name]
            print(f"  🔗 同一实例: {same_instance}")
        else:
            print(f"  ❌ {akshare_name} 不存在")
            
    except Exception as e:
        print(f"  ❌ 错误: {str(e)}")

🔄 多名称访问模式演示
同一个fetcher可以通过业务名称和akshare方法名访问:


📋 上交所总貌:
  📈 sse_summary: (8, 4)
  📈 stock_sse_summary: (8, 4)
  🔗 同一实例: True

📋 深交所总貌:
  📈 szse_summary: (14, 5)
  📈 stock_szse_summary: (14, 5)
  🔗 同一实例: True

📋 融资融券:
  📈 margin_financing: (2001, 8)
  📈 stock_margin_sse: (2001, 8)
  🔗 同一实例: True

📋 龙虎榜:
  📈 lhb: (637, 21)
  📈 stock_lhb_detail_em: (637, 21)
  🔗 同一实例: True


## 5. 数据统计总览

In [22]:
print("📊 MarketData所有Fetchers数据统计总览")
print("=" * 70)

# 统计所有fetcher的数据
stats = []
unique_fetchers = set()  # 去重，因为有多名称的fetcher

# 收集唯一的fetcher实例
for name, fetcher in market.fetchers.items():
    unique_fetchers.add(fetcher)

print(f"总计唯一fetchers: {len(unique_fetchers)}")
print(f"总计访问名称: {len(market.fetchers)}")
print("\n详细统计:")
print("-" * 70)

for i, name in enumerate(sorted(market.fetchers.keys()), 1):
    try:
        data = market.get_data(name)
        if not data.empty:
            status = f"✅ {data.shape[0]}行 x {data.shape[1]}列"
        else:
            status = "❌ 空数据"
    except Exception as e:
        status = f"❌ 错误: {str(e)[:30]}..."
    
    print(f"{i:2d}. {name:<35} {status}")

print("\n" + "=" * 70)
print("✨ 数据获取完成！")

📊 MarketData所有Fetchers数据统计总览
总计唯一fetchers: 8
总计访问名称: 17

详细统计:
----------------------------------------------------------------------
 1. lhb                                 ✅ 637行 x 21列
 2. margin_financing                    ✅ 2001行 x 8列
 3. northbound_holdings                 ✅ 1336行 x 16列
 4. sse_deal_daily                      ✅ 9行 x 6列
 5. sse_summary                         ✅ 8行 x 4列
 6. stock_hsgt_hold_stock_em            ✅ 1336行 x 16列
 7. stock_lhb_detail_em                 ✅ 637行 x 21列
 8. stock_margin_sse                    ✅ 2001行 x 8列
 9. stock_margin_szse                   ✅ 2001行 x 8列
10. stock_sse_deal_daily                ✅ 9行 x 6列
11. stock_sse_summary                   ✅ 8行 x 4列
12. stock_szse_area_summary             ✅ 34行 x 7列
13. stock_szse_sector_summary           ✅ 20行 x 9列
14. stock_szse_summary                  ✅ 14行 x 5列
15. szse_area_summary                   ✅ 34行 x 7列
16. szse_sector_summary                 ✅ 20行 x 9列
17. szse_summary                      

## 6. 参数化访问演示

In [23]:
print("🔧 参数化访问演示")
print("展示不同参数设置下的MarketData行为\n")

# 创建不同参数的MarketData实例
test_configs = [
    {'date': '2024-08-30', 'symbol': '当月', 'desc': '默认参数'},
    {'date': '2024-07-15', 'symbol': '本月', 'desc': '自定义日期和符号'},
    {'date': '202407', 'symbol': '当年', 'desc': 'YYYYMM格式日期'},
]

for config in test_configs:
    print(f"\n📋 {config['desc']}:")
    print(f"   输入参数: date='{config['date']}', symbol='{config['symbol']}'")
    
    try:
        test_market = MarketData(date=config['date'], symbol=config['symbol'])
        print(f"   处理结果:")
        print(f"     - date: {test_market.date}")
        print(f"     - date_yyyymm: {test_market.date_yyyymm}")
        print(f"     - date_yyyymmdd: {test_market.date_yyyymmdd}")
        print(f"     - symbol: {test_market.symbol}")
        print(f"     - fetchers数量: {len(test_market.fetchers)}")
        
        # 测试一个简单的fetcher
        if 'sse_summary' in test_market.fetchers:
            data = test_market.sse_summary
            print(f"     - sse_summary数据: {data.shape if not data.empty else '空数据'}")
        
    except Exception as e:
        print(f"   ❌ 错误: {str(e)}")

🔧 参数化访问演示
展示不同参数设置下的MarketData行为


📋 默认参数:
   输入参数: date='2024-08-30', symbol='当月'
   处理结果:
     - date: 20240830
     - date_yyyymm: 202408
     - date_yyyymmdd: 20240830
     - symbol: 当月
     - fetchers数量: 17
     - sse_summary数据: (8, 4)

📋 自定义日期和符号:
   输入参数: date='2024-07-15', symbol='本月'
   处理结果:
     - date: 20240715
     - date_yyyymm: 202407
     - date_yyyymmdd: 20240715
     - symbol: 本月
     - fetchers数量: 17
     - sse_summary数据: (8, 4)

📋 YYYYMM格式日期:
   输入参数: date='202407', symbol='当年'
   处理结果:
     - date: 20240701
     - date_yyyymm: 202407
     - date_yyyymmdd: 20240701
     - symbol: 当年
     - fetchers数量: 17
     - sse_summary数据: (8, 4)


## 7. 使用建议

### 📚 MarketData使用指南

#### 1. 创建实例
```python
# 使用默认参数（当前日期）
market = MarketData()

# 自定义日期和符号
market = MarketData(date='2024-08-30', symbol='当月')
```

#### 2. 数据访问方式
```python
# 方式1: 属性访问
data = market.sse_summary

# 方式2: 方法访问
data = market.get_data('sse_summary')

# 方式3: 使用akshare方法名
data = market.get_data('stock_sse_summary')
```

#### 3. 主要数据类型
- **市场情绪**: `market_sentiment` / `index_news_sentiment_scope`
- **融资融券**: `margin_financing` / `stock_margin_sse` / `stock_margin_szse`
- **北向资金**: `northbound_holdings`
- **龙虎榜**: `lhb` / `stock_lhb_detail_em`
- **上交所数据**: `sse_summary`, `sse_deal_daily`
- **深交所数据**: `szse_summary`, `szse_area_summary`, `szse_sector_summary`

#### 4. 日期格式支持
- `YYYY-MM-DD`: 2024-08-30
- `YYYYMMDD`: 20240830
- `YYYYMM`: 202408

系统会自动转换为API所需的格式。